In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
import folium
import random
import pandas as pd

In [3]:
availabilities = pd.read_csv('../data/nextbike_availability.csv')
availabilities

,nextbike_availability_at_station,nextbike_availability_vehicle_id,nextbike_availability_geometry,nextbike_availability_status,nextbike_availability_valid_from,nextbike_availability_valid_till
0,False,6045,POINT(6.923261165618896 50.92481231689453),available,2022-02-07 12:01:00,2022-02-07 12:31:00
1,False,5987,POINT(6.981607913970947 50.94987869262695),available,2022-03-12 15:01:00,2022-03-12 15:01:00
2,False,5987,POINT(6.981634140014648 50.94987869262695),available,2022-03-12 15:06:00,2022-03-12 16:31:00
3,False,5987,POINT(6.956296920776367 50.94599533081055),available,2022-03-12 16:51:00,2022-03-12 16:56:00
4,False,5987,POINT(6.956347942352295 50.94609069824219),available,2022-03-12 17:01:00,2022-03-12 17:01:00
...,...,...,...,...,...,...
8329914,False,28428,POINT(6.954068183898926 50.94620132446289),available,2023-08-25 11:11:00,2023-08-25 12:26:00
8329915,False,28428,POINT(6.953924179077148 50.9462776184082),available,2023-08-25 12:31:00,2023-08-25 12:31:00
8329916,False,28428,POINT(6.954151153564453 50.94615936279297),available,2023-08-25 12:36:00,2023-08-25 17:11:00
8329917,False,28428,POINT(6.924293041229248 50.9334716796875),available,2023-08-25 17:31:00,2023-08-25 17:51:00


In [4]:
# remove "nextbike_availability_" prefix
availabilities.columns = availabilities.columns.str.replace('nextbike_availability_', '')

In [5]:
availabilities["lon"] = (
    availabilities.geometry.apply(lambda x: x.split(" ")[0])
    .apply(lambda x: x.split("(")[1])
    .astype(float)
)
availabilities["lat"] = (
    availabilities.geometry.apply(lambda x: x.split(" ")[1])
    .apply(lambda x: x.split(")")[0])
    .astype(float)
)


In [6]:
availabilities["valid_from"] = pd.to_datetime(availabilities["valid_from"])
availabilities["valid_till"] = pd.to_datetime(availabilities["valid_till"])

In [19]:
buckets = pd.date_range(availabilities.valid_from.min(), availabilities.valid_till.max(), freq='1H')
buckets

DatetimeIndex(['2022-01-15 00:01:00', '2022-01-15 01:01:00',
               '2022-01-15 02:01:00', '2022-01-15 03:01:00',
               '2022-01-15 04:01:00', '2022-01-15 05:01:00',
               '2022-01-15 06:01:00', '2022-01-15 07:01:00',
               '2022-01-15 08:01:00', '2022-01-15 09:01:00',
               ...
               '2023-08-31 14:01:00', '2023-08-31 15:01:00',
               '2023-08-31 16:01:00', '2023-08-31 17:01:00',
               '2023-08-31 18:01:00', '2023-08-31 19:01:00',
               '2023-08-31 20:01:00', '2023-08-31 21:01:00',
               '2023-08-31 22:01:00', '2023-08-31 23:01:00'],
              dtype='datetime64[ns]', length=14256, freq='H')

In [20]:
from package import h3

In [21]:
availabilities = h3.add_h3_cell_id_to_df_with_batching(
    availabilities, 8, n_batches=16 * 10
)


/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [24]:
import multiprocessing
from tqdm.auto import tqdm
from package import key

all_hex_ids = availabilities.h3_cell_id.unique()


def count_bikes_at_time(time_point):
    locations_at_time = get_locations_at_time(availabilities, time_point)
    
    n_bikes_per_hex = locations_at_time.groupby("h3_cell_id").size()
    n_bikes_per_hex = n_bikes_per_hex.reindex(all_hex_ids, fill_value=0)
    n_bikes_per_hex.name = time_point
    return n_bikes_per_hex


with multiprocessing.Pool(key.DEFAULT_N_PROCESSES) as pool:
    n_bikes_per_hex_per_time = list(
        tqdm(pool.imap(count_bikes_at_time, buckets), total=len(buckets))
    )


  0%|          | 0/14256 [00:00<?, ?it/s]

In [26]:
n_bikes_per_hex_per_time = pd.DataFrame(n_bikes_per_hex_per_time)
n_bikes_per_hex_per_time.index.name = "time"


In [28]:
n_bikes_per_hex_per_time

h3_cell_id,881fa199abfffff,881fa18a4dfffff,881fa1999dfffff,881fa19999fffff,881fa199a1fffff,881fa199d3fffff,881fa199c3fffff,881fa199c1fffff,881fa199e1fffff,881fa199a9fffff,...,881fa18027fffff,881fa18f2dfffff,881fa18305fffff,881fa18133fffff,881fa1812dfffff,881fa18855fffff,881fa19a85fffff,881fa18909fffff,881fa19ab1fffff,881fa181e5fffff
time,,,,,,,,,,,,,,,,,,,,,
2022-01-15 00:01:00,8,3,35,36,17,11,26,21,5,24,...,0,0,0,0,0,0,0,0,0,0
2022-01-15 01:01:00,7,3,35,36,20,10,27,17,5,23,...,0,0,0,0,0,0,0,0,0,0
2022-01-15 02:01:00,9,3,36,33,20,10,27,17,4,28,...,0,0,0,0,0,0,0,0,0,0
2022-01-15 03:01:00,10,3,34,33,23,9,29,19,3,28,...,0,0,0,0,0,0,0,0,0,0
2022-01-15 04:01:00,12,3,34,34,23,9,29,21,3,29,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31 19:01:00,15,7,18,15,8,6,16,7,4,5,...,1,3,2,2,6,3,5,4,3,0
2023-08-31 20:01:00,14,8,15,15,13,13,22,10,2,8,...,1,3,2,2,5,3,5,4,3,0
2023-08-31 21:01:00,15,6,17,13,23,14,22,9,4,12,...,1,3,2,2,5,3,5,4,3,0


In [42]:
import pandas as pd
from sklearn_extra.cluster import KMedoids

# Number of clusters you want
n_clusters = 4

model = KMedoids(n_clusters=n_clusters, random_state=4711)
model.fit(n_bikes_per_hex_per_time)

# n_bikes_per_hex_per_time["cluster"] = model.labels_


In [53]:
medoids = n_bikes_per_hex_per_time.iloc[model.medoid_indices_]

In [60]:
cologne_center = [50.938361, 6.959974]
m = folium.Map(location=cologne_center, zoom_start=12)
h3.plot_h3_cells_on_folium(medoids.iloc[0].to_dict(), m)
m

In [82]:
time = medoids.index[3]
locations_at_time = get_locations_at_time(availabilities, time)
time

Timestamp('2022-07-18 06:01:00')

In [83]:
# center in cologne
m = folium.Map(location=[50.938361, 6.959974], zoom_start=13)

bbox = [50.888361, 6.889974, 50.988361, 6.999974]

folium.Rectangle(
	bounds=[[bbox[0], bbox[1]], [bbox[2], bbox[3]]],
	color='red',
	fill=False,
).add_to(m)


for point in locations_at_time[['lat', 'lon']].values:
	folium.CircleMarker(
		location=point,
		radius=2,
		color='#000000',
		fill=True,
		fill_color='#000000'
	).add_to(m)


m

In [88]:
locations_at_time[["lat", "lon"]].to_csv("../data/bicycle_locations.csv", index=False)